<a href="https://colab.research.google.com/github/MHSITC/hello-world/blob/main/OpenVoice_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/OpenVoice
%cd /content/OpenVoice

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/OpenVoice/resolve/main/checkpoints_1226.zip -d /content -o checkpoints_1226.zip
!unzip /content/checkpoints_1226.zip

!pip install -q gradio==3.50.2 langid faster-whisper whisper-timestamped unidecode eng-to-ipa pypinyin cn2an

!python openvoice_app.py --share

/content
Cloning into 'OpenVoice'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211 (from 1)
Receiving objects: 100% (211/211), 2.28 MiB | 25.93 MiB/s, done.
Resolving deltas: 100% (94/94), done.
/content/OpenVoice
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1

In [ ]:
# --- Setup Environment ---
!git clone https://github.com/myshell-ai/OpenVoice.git
%cd OpenVoice
!pip install -q -r requirements.txt
!pip install -q "unidic2"   # for Japanese if needed, skip if English only

# Use GPU if available
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Running on device:", device)

from openvoice import TTS, Speaker
# (Check actual class names in repo; adjust as needed)

# --- Load Model ---
tts = TTS(device=device)
speaker_ids = {
    "A": tts.add_speaker("speakerA_reference.wav"),  # optional: provide a short reference audio or use default
    "B": tts.add_speaker("speakerB_reference.wav")
}

# --- Define Dialogue Script ---
dialogue = [
    ("A", "Hey Dua, do you have any plans for Saturday morning?"),
    ("B", "Hi Ed! Not really. I usually go to the gym on Saturdays. Do you want to come with me?"),
    ("A", "mmm… the gym? I can’t stand going to the gym — it’s not my thing. I prefer playing my guitar at home. I’m crazy about writing new songs!"),
    ("B", "Oh, come on! You’re always writing songs!"),
    ("A", "You’re right! But maybe we can do something we both enjoy. Do you like walking? We can walk and listen to music!"),
    ("B", "That actually sounds fun! A bit of exercise and some music — perfect mix!"),
    ("A", "Perfect! Saturday morning: walk and music. See you!"),
    ("B", "Bye‑bye!")
]

# --- Synthesize Each Line & Concatenate ---
from pydub import AudioSegment

output_audio = AudioSegment.silent(duration=0)
for speaker_key, text in dialogue:
    sid = speaker_ids[speaker_key]
    wav = tts.synthesize(text=text, speaker_id=sid, device=device)  # returns a numpy array or file path
    # Convert to AudioSegment
    seg = AudioSegment(
        wav.tobytes(),
        frame_rate=tts.sample_rate,
        sample_width=wav.dtype.itemsize,
        channels=1
    )
    # Optional: add small pause between lines
    seg = seg + AudioSegment.silent(duration=300)  # 300 ms pause
    output_audio += seg

# --- Export Result ---
output_audio.export("dialogue_output.mp3", format="mp3")
print("✅ Exported as dialogue_output.mp3")
